## Weather API

In [19]:
from datetime import datetime, timedelta
import requests
import json
import config
import pandas as pd

In [20]:
weather_api_key = config.api_key

weather_test = requests.get("http://api.weatherapi.com/v1/current.json", params={"key": weather_api_key, "q": "London"})
weather_json_test = json.loads(weather_test.text)
print(weather_json_test.get("current").get("last_updated"))

2024-04-11 15:00


In [21]:
def get_weather_data(location, wtype, interval, apikey):
    columns = ["date", "max_temp", "min_temp", "avg_temp", "total_precipitation", "type"]
    loc_lower = location.lower()
    wtype_lower = wtype.lower()

    params = {"key": apikey,
              "q": loc_lower,
              }

    if wtype_lower == "forecast":
        params["days"] = interval
        weather = requests.get(f"https://api.weatherapi.com/v1/{wtype_lower}.json", params=params)
        weather_json = json.loads(weather.text)

        weather_df = convert_weather_data_to_df(weather_json, columns, wtype_lower)
    elif wtype_lower == "history":
        end_dt = datetime.now().strftime("%Y-%m-%d")
        start_dt = (datetime.now() - timedelta(days=interval)).strftime("%Y-%m-%d")
        params["dt"] = start_dt
        params["end_dt"] = end_dt
    
        weather = requests.get(f"https://api.weatherapi.com/v1/{wtype_lower}.json", params=params)
        weather_json = json.loads(weather.text)

        weather_df = convert_weather_data_to_df(weather_json, columns, wtype_lower)
    else:
        raise ValueError("Wrong weather type")

    return weather_df

In [22]:
def convert_weather_data_to_df(weather_json, columns, wtype_lower):
    if 'forecast' in weather_json and 'forecastday' in weather_json['forecast']:
        history_data = weather_json['forecast']['forecastday']
        weather_data = []
        for history in history_data:
            date = history['date']
            max_temp = history['day']['maxtemp_c']
            min_temp = history['day']['mintemp_c']
            avg_temp = history['day']['avgtemp_c']
            total_precipitation = history['day']['totalprecip_mm']
            weather_type = wtype_lower
            weather_data.append([date, max_temp, min_temp, avg_temp, total_precipitation, weather_type])

        return pd.DataFrame(weather_data, columns=columns)
    else:
        print("No forecast data found.")

In [23]:
print(get_weather_data("LONDON", "FORECAST", 3, weather_api_key))

         date  max_temp  min_temp  avg_temp  total_precipitation      type
0  2024-04-11      16.6      11.7      13.5                 0.21  forecast
1  2024-04-12      19.3      10.9      14.7                 0.00  forecast
2  2024-04-13      19.1      10.2      14.6                 0.01  forecast


In [24]:
print(get_weather_data("London", "History", 3, weather_api_key))

         date  max_temp  min_temp  avg_temp  total_precipitation     type
0  2024-04-08      17.2       9.0      12.6                 1.70  history
1  2024-04-09      11.6       6.9       8.8                 0.60  history
2  2024-04-10      14.0       5.2       9.4                 0.02  history
3  2024-04-11      19.2      11.7      14.5                 0.09  history
